In [93]:
import pandas as pd
import seaborn as sns
import pickle

import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression, LinearRegression, Lasso
from sklearn.metrics import mean_squared_error

* Download the Data using the command line terminal wget
* Referal to learn with Machine Learning Zoomcamp 2:23
* Read Data and understanding different columns of the data using df.head and data dictionary 3:06
* * 
* Calulate the duration of each trip fropoff - pickup time.
* * convert value to pandas date time and calculate the difference in value
* * Prprocess the created duration column into minutes
* * Visuale distribution and data describe of duration 9:50
* * Filter duration that is within 1 and 60 mins  13:10
* * Specify catergorical and numeric variables
* if you do not have pyarrow installed or fastparquet, you will have to install one
** 

In [94]:
# Specify the location of the file
green_trip_data_Jan_2021 = 'data/green_tripdata_2021-01.parquet'
green_trip_data_Feb_2021 = 'data/green_tripdata_2021-02.parquet'

fhv_trip_data_Jan_2021 = 'data/fhv_tripdata_2021-01.parquet'
fhv_trip_data_Feb_2021 = 'data/fhv_tripdata_2021-02.parquet'

In [95]:
def read_dataframe(filename, pickup_time, drop_off_time, pu_id, do_id):
    df = pd.read_parquet(filename)
    df['duration'] = pd.to_datetime(df[drop_off_time]) - pd.to_datetime(df[pickup_time])  # calculate duration and convert to minutes
    df['duration'] = df['duration'].apply(lambda td: td.total_seconds()/60)
 
    df = df[((df.duration>=1) & (df.duration<=60))]       # filter to only duration within and less than hour



    categorical = [pu_id, do_id]

    df[categorical] = df_Jan[categorical].fillna(-1)
    df[categorical] = df[categorical].astype(str)
    return df

## Q1 Read the data for January. 
How many records are there?

In [96]:
df_Jan = pd.read_parquet(fhv_trip_data_Jan_2021)
df_Jan.shape

(1154112, 7)

## Q2. Computing duration
Now let's compute the `duration` variable. It should contain the duration of a ride in minutes.

What's the average trip duration in January?



In [97]:
df_Jan['duration'] = pd.to_datetime(df_Jan['dropOff_datetime'])- pd.to_datetime(df_Jan['pickup_datetime'])
df_Jan['duration'] = df_Jan['duration'].apply(lambda x: x.total_seconds()/60)
df_Jan['duration'].mean()

19.1672240937939

## Data preparation
Check the distribution of the duration variable. There are some outliners.

Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

How many records did you drop?



In [98]:
df_Jan = df_Jan[((df_Jan.duration>=1) & (df_Jan.duration<=60))]

## Q3. Missing values
The features we'll use for our model are the pickup and dropoff location IDs.

But they have a lot of missing values there. Let's replace them with "-1"

What's the factions of missing values for the pickup location ID? (Or the fraction of "-1"s after you filled the NAs)

In [99]:
train_col = ['PUlocationID', 'DOlocationID']
df_Jan[train_col].isnull().sum()/df_Jan.shape[0]

PUlocationID    0.835273
DOlocationID    0.133270
dtype: float64

In [100]:
# fill missing value 
df_Jan[train_col] = df_Jan[train_col].fillna(-1)
df_Jan[train_col] = df_Jan[train_col].astype('str')

## Q4. One-hot encoding
Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

* Turn the dataframe into a list of dictionaries
* Fit a dictionary vectorizer
* Get a feature matrix from it

In [109]:
dict_train = df_Jan[train_col].to_dict(orient='records')
dv = DictVectorizer()
X_train = dv.fit_transform(dict_train) #this returns a sparse cmr matrix
Y_train = df_Jan['duration'].values

In [102]:
X_train.shape

(1109826, 525)

## Q5. Training a model
Now let's use the feature matrix from the previous step to train a model.

* Train a plain linear regression model with default parameters
* Calculate the RMSE of the model on the training data

In [103]:
lr = LinearRegression()
lr.fit(X_train, Y_train)

Y_pred = lr.predict(X_train) # Make prediciton on the train
mean_squared_error(Y_train, Y_pred, squared=False)

10.5285191072072

## Q6. Evaluating the model
Now let's apply this model to the validation dataset (Feb 2021).

What's the RMSE on validation?

In [110]:
df_train = read_dataframe(fhv_trip_data_Jan_2021,'pickup_datetime',  'dropOff_datetime', pu_id='PUlocationID', do_id='DOlocationID')
df_val = read_dataframe(fhv_trip_data_Feb_2021,'pickup_datetime',  'dropOff_datetime', pu_id='PUlocationID', do_id='DOlocationID')

# dv = DictVectorizer()
train_dicts = df_val[train_col].to_dict(orient='records')
X_val = dv.transform(train_dicts)


In [111]:
X_val.shape

(990113, 525)

In [112]:
Y_pred = lr.predict(X_val) # Make prediciton on the train
mean_squared_error(df_val.duration.values, Y_pred, squared=False)

12.923643749124812

In [44]:
# ! conda install -c conda-forge fastparquet
# ! conda install -c conda-forge pyarrow


In [97]:
with open ('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr),f_out)